In [456]:
# !pip install altair 
# !pip install altair_saver
# !npm install -g vega-lite vega-cli canvas
# !pip install vl-convert-python

In [457]:
# !apt update
# !apt install ttf-mscorefonts-installer -y
# !apt reinstall fontconfig fontconfig-config libfontconfig1 -y

In [458]:
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Black.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Bold.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Book.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Medium.otf -P /usr/local/share/fonts

In [459]:
# !fc-cache -f

In [460]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import urllib.parse

In [461]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [462]:
SAVE = True
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [463]:
%%capture pwd
!pwd

In [464]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
uid=urllib.parse.quote(uid)
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
mo=0.5
height = config["height"]
width = config["width"]
uid, height, width

('how-has-the-pandemic-affected-young-peoples-employment', 300, 500)

In [465]:
def save(df, f, LOCAL):
    fc = eco_git_path + f + ".csv"
    df.to_csv("data/" + f + ".csv")
    f += local_suffix
    open("visualisation/" + f + ".html", "w").write(
        vega_embed.replace(
            "JSON_PATH", fc.replace("/data/", "/visualisation/").replace(".csv", ".json")
        )
    )
    if LOCAL:
        fc = df
    
    from IPython.display import display, HTML
    display(HTML(df.head().to_html()))
    
    readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
    return readme, f, fc

In [466]:
def area(base,color,opacity=1):
    return base.mark_area(opacity=opacity,
    interpolate="monotone",
    line={'color':color},
    color=alt.Gradient(
        gradient='linear',
        stops=[alt.GradientStop(color='#ffffff00', offset=0.2),
               alt.GradientStop(color=color, offset=0.8)],
        x1=1, #0.8
        y1=1,
        x2=1,
        y2=0
        )
    )

# Fig 1

## a

In [467]:
df = pd.read_excel("raw/1.xlsx")

In [468]:
readme, f, fc = save(df,"fig1a_worklessness",LOCAL)

,myr,Unemployed,Inactive (student),Inactive (other),Employed and working zero hours
0,2019-01-01,-0.677104,1.842883,-0.039511,-0.769292
1,2019-04-01,-0.103328,1.599851,0.306677,-0.605997
2,2019-07-01,1.357871,-1.688179,1.463310,0.002209
3,2019-10-01,0.000000,0.000000,0.000000,0.000000
4,2020-01-01,0.212764,0.820026,-0.041386,1.189065


In [469]:
xaxis = alt.Chart(pd.DataFrame([{'x':'2019-01-01','y':0},{'x':'2022-04-01','y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="Percentage point change from 2019 Q4",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)
yaxis=alt.Chart(pd.DataFrame([{'x':'2020-01-01','y':-4},{'x':'2020-01-01','y':14}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(x='x:T',y='y:Q')
ylabel=alt.Chart(pd.DataFrame([{'x':'2019-01-01','y':-3.2,'t':'↓ decrease'},{'x':'2019-01-01','y':13,'t':'↑ increase'}]))\
    .mark_text(color=colors["eco-gray"],opacity=mo+0.1,align='left',dx=5).encode(x='x:T',y='y:Q',text='t:N')
ylabel2=alt.Chart(pd.DataFrame([{'x':'2022-04-01','y':14,'t':'Aged 16-24'}]))\
    .mark_text(color=colors["eco-gray"],opacity=mo+0.3,align='right',dx=-5,dy=-12,fontSize=12,fontWeight='bold').encode(x='x:T',y='y:Q',text='t:N')

base = alt.Chart(fc).encode(x='myr:T')
line1=area(base,colors['eco-dot']).encode(y='Unemployed:Q')
line2=area(base,colors['eco-light-blue']).encode(y='Inactive (student):Q')
line3=area(base,colors['eco-blue']).encode(y='Inactive (other):Q')
line4=area(base,colors['eco-turquiose']).encode(y='Employed and working zero hours:Q')

label1=line1.mark_text(color=colors['eco-dot'],align='left',dx=5,fontSize=12).encode(text='t:N').transform_calculate(t='"Unemployed"').transform_filter('datum.myr>toDate("2022-03-01")')
label2=line2.mark_text(color=colors['eco-light-blue'],align='left',dx=5,fontSize=12).encode(text='t:N').transform_calculate(t='"Inactive (student)"').transform_filter('datum.myr>toDate("2022-03-01")')
label3=line3.mark_text(color=colors['eco-blue'],align='left',dx=5,fontSize=12).encode(text='t:N').transform_calculate(t='"Inactive (other)"').transform_filter('datum.myr>toDate("2022-03-01")')
label4=line4.mark_text(color=colors['eco-turquiose'],align='left',dx=5,fontSize=12).encode(text='t:N').transform_calculate(t='"Working zero hours"').transform_filter('datum.myr>toDate("2022-03-01")')
color=colors['eco-turquiose']
layer1 = (
    (xaxis+yaxis+ylabel+ylabel2+line4+line2+line3+line1+label1+label2+label3+label4).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    open("README.md", "w").write(readme)

logo=alt.Chart(pd.DataFrame([{"x": '2022-04-01', "y": 14, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=105).encode(x='x:T',y='y:Q',url='img:N')
ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
    .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")

print(f+'\n')
layer1

fig1a_worklessness



alt.LayerChart(...)

## b

In [470]:
df = pd.read_excel("raw/1b.xlsx")

In [471]:
readme, f, fc = save(df,"fig1b_worklessness25",LOCAL)

,myr,Unemployed,Inactive (student),Inactive (other),Employed and working zero hours
0,2019-01-01,0.250514,0.015727,0.236362,-2.202801
1,2019-04-01,0.155684,0.037781,0.210358,-1.487029
2,2019-07-01,0.091337,-0.057789,0.262488,0.732846
3,2019-10-01,0.000000,0.000000,0.000000,0.000000
4,2020-01-01,0.245057,-0.050963,-0.003825,-0.659347


In [472]:
xaxis = alt.Chart(pd.DataFrame([{'x':'2019-01-01','y':0},{'x':'2022-04-01','y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="Percentage point change from 2019 Q4",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)
yaxis=alt.Chart(pd.DataFrame([{'x':'2020-01-01','y':-4},{'x':'2020-01-01','y':14}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(x='x:T',y='y:Q')
ylabel=alt.Chart(pd.DataFrame([{'x':'2019-01-01','y':-3.2,'t':'↓ decrease'},{'x':'2019-01-01','y':13,'t':'↑ increase'}]))\
    .mark_text(color=colors["eco-gray"],opacity=mo+0.1,align='left',dx=5).encode(x='x:T',y='y:Q',text='t:N')
ylabel2=alt.Chart(pd.DataFrame([{'x':'2022-04-01','y':14,'t':'Aged 25-64'}]))\
    .mark_text(color=colors["eco-gray"],opacity=mo+0.3,align='right',dx=-5,dy=-12,fontSize=12,fontWeight='bold').encode(x='x:T',y='y:Q',text='t:N')

base = alt.Chart(fc).encode(x='myr:T')
line1=area(base,colors['eco-dot']).encode(y='Unemployed:Q')
line2=area(base,colors['eco-light-blue']).encode(y='Inactive (student):Q')
line3=area(base,colors['eco-blue']).encode(y='Inactive (other):Q')
line4=area(base,colors['eco-turquiose']).encode(y='Employed and working zero hours:Q')

label1=line1.mark_text(color=colors['eco-dot'],align='left',dx=5,fontSize=12).encode(text='t:N').transform_calculate(t='"Unemployed"').transform_filter('datum.myr>toDate("2022-03-01")')
label2=line2.mark_text(color=colors['eco-light-blue'],align='left',dx=5,dy=12,fontSize=12).encode(text='t:N').transform_calculate(t='"Inactive (student)"').transform_filter('datum.myr>toDate("2022-03-01")')
label3=line3.mark_text(color=colors['eco-blue'],align='left',dx=5,fontSize=12).encode(text='t:N').transform_calculate(t='"Inactive (other)"').transform_filter('datum.myr>toDate("2022-03-01")')
label4=line4.mark_text(color=colors['eco-turquiose'],align='left',dx=5,fontSize=12).encode(text='t:N').transform_calculate(t='"Working zero hours"').transform_filter('datum.myr>toDate("2022-03-01")')
color=colors['eco-turquiose']
layer1 = (
    (xaxis+yaxis+ylabel+ylabel2+line4+line2+line3+line1+label1+label2+label3+label4).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    open("README.md", "w").write(readme)

logo=alt.Chart(pd.DataFrame([{"x": '2022-04-01', "y": 14, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=105).encode(x='x:T',y='y:Q',url='img:N')
ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
    .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")

print(f+'\n')
layer1

fig1b_worklessness25



alt.LayerChart(...)

# Fig 2

In [473]:
df = pd.read_excel("raw/2.xlsx")
df=df[['myr','Vacancies']]

In [474]:
readme, f, fc = save(df,"fig2_vacancies",LOCAL)

,myr,Vacancies
0,2001-04-01,669
1,2001-05-01,689
2,2001-06-01,692
3,2001-07-01,670
4,2001-08-01,667


In [475]:
xaxis = alt.Chart(pd.DataFrame([{'x':'2001-04-01','y':0},{'x':'2022-10-01','y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="Vacancies (thousands)",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)
ylabel=alt.Chart(pd.DataFrame([{'x':'2007-10-01','y':0,'t':'Global Financial Crisis'}]))\
    .mark_text(color=colors["eco-gray"],opacity=mo+0.1,align='left',dy=-10,dx=5).encode(x='x:T',y='y:Q',text='t:N')
ylabel2=alt.Chart(pd.DataFrame([{'x':'2020-03-01','y':0,'t':'COVID-19 →'}]))\
    .mark_text(color=colors["eco-gray"],opacity=mo+0.3,align='left',dy=-10,dx=5).encode(x='x:T',y='y:Q',text='t:N')

base = alt.Chart(fc).encode(x='myr:T')
line1=area(base,colors['eco-turquiose']).encode(y='Vacancies:Q')
area1 = alt.Chart(pd.DataFrame([{'x':'2007-11-01','y':0,'y2':1400},{'x':'2011-10-01','y':0,'y2':1400}])).mark_area(color=colors["eco-gray"],opacity=0.1)\
    .encode(x='x:T',y='y:Q',y2='y2:Q')
area2 = alt.Chart(pd.DataFrame([{'x':'2020-03-01','y':0,'y2':1400},{'x':'2022-10-01','y':0,'y2':1400}])).mark_area(color=colors["eco-gray"],opacity=0.1)\
    .encode(x='x:T',y='y:Q',y2='y2:Q')
area2e=alt.Chart(pd.DataFrame([{'x':'2022-10-01','y':0,'y2':1400},{'x':'2023-10-01','y':0,'y2':1400}])).mark_area(
    opacity=0.1,
    color=alt.Gradient(
        gradient='linear',
        stops=[alt.GradientStop(color='#ffffff00', offset=0),
               alt.GradientStop(color=colors["eco-gray"], offset=1)],
        x1=1,
        y1=1,
        x2=0,
        y2=1
        )
    ).encode(x='x:T',y='y:Q',y2='y2:Q')

color=colors['eco-turquiose']
layer1 = (
    (xaxis+line1+area1+area2+area2e+ylabel+ylabel2).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    open("README.md", "w").write(readme)

logo=alt.Chart(pd.DataFrame([{"x": '2023-10-01', "y": 1400, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=5).encode(x='x:T',y='y:Q',url='img:N')
ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
    .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")

print(f+'\n')
layer1

fig2_vacancies



alt.LayerChart(...)

# Fig 3

In [476]:
df = pd.read_excel("raw/3.xlsx",usecols='A:D')
df*=100.0
df['cohort']/=100.0
df['dt']=df['cohort'].astype(int).astype(str)+'-01-01'

In [477]:
readme, f, fc = save(df,"fig3_graduation",LOCAL)

,cohort,3-6 months,9-12 months,1-2 years,dt
0,2004.0,68.126929,3.258771,5.497199,2004-01-01
1,2005.0,65.413105,2.539098,7.967371,2005-01-01
2,2006.0,65.181446,2.768296,7.958484,2006-01-01
3,2007.0,66.974419,4.280466,1.532614,2007-01-01
4,2008.0,66.015023,-1.168317,4.167157,2008-01-01


In [478]:
xaxis = alt.Chart(pd.DataFrame([{'x':'2004-01-01','y':0},{'x':'2021-01-01','y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "dt:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="Employment rates by graduation cohort (%)",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        scale=alt.Scale(domain=[0, 100],clamp=True),
        
    )
)

base = alt.Chart(fc).encode(x='dt:T')
line1=area(base,colors['eco-dot']).encode(y='1-2 years:Q')
line2=area(base,colors['eco-light-blue']).encode(y='9-12 months:Q')
line3=area(base,colors['eco-turquiose']).encode(y='3-6 months:Q')
line4=base.mark_line(color=colors['eco-blue'],interpolate='monotone',strokeDash=[5,5],opacity=0.7).encode(y='y:Q').transform_calculate(y='datum["1-2 years"]+datum["9-12 months"]+datum["3-6 months"]')

label1=line1.mark_text(color=colors['eco-dot'],align='left',dx=30,dy=-10,fontSize=12).encode(text='t:N').transform_calculate(t='"1-2 years"').transform_filter('datum.dt>toDate("2019-03-01")')
label2=line2.mark_text(color=colors['eco-light-blue'],align='left',dx=5,fontSize=12).encode(text='t:N').transform_calculate(t='"9-12 months"').transform_filter('datum.dt>toDate("2020-03-01")')
label3=line3.mark_text(color=colors['eco-turquiose'],align='left',dx=5,fontSize=12).encode(text='t:N').transform_calculate(t='"3-6 months"').transform_filter('datum.dt>toDate("2020-03-01")')
label4=line4.mark_text(color=colors['eco-blue'],align='left',dx=5,fontSize=12).encode(text='t:N').transform_calculate(t='"Cumulative"').transform_filter('datum.dt>toDate("2020-03-01")')
color=colors['eco-turquiose']
layer1 = (
    (xaxis+line1+line2+line3+line4+label1+label2+label3+label4).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    open("README.md", "w").write(readme)

logo=alt.Chart(pd.DataFrame([{"x": '2021-01-01', "y": 100, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=75).encode(x='x:T',y='y:Q',url='img:N')
ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
    .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")

print(f+'\n')
layer1

WARN Infinite extent for field "dt": [Infinity, -Infinity]
WARN Infinite extent for field "dt": [Infinity, -Infinity]
WARN Infinite extent for field "dt": [Infinity, -Infinity]
WARN Infinite extent for field "dt": [Infinity, -Infinity]
WARN Infinite extent for field "dt": [Infinity, -Infinity]
WARN Infinite extent for field "dt": [Infinity, -Infinity]
WARN Infinite extent for field "dt": [Infinity, -Infinity]
WARN Infinite extent for field "dt": [Infinity, -Infinity]


fig3_graduation



alt.LayerChart(...)

# Fig 4

In [479]:
df = pd.read_excel("raw/4.xlsx").dropna(how='any',axis=1)
df*=100.0
df['cohort']/=100.0
df['dt']=df['cohort'].astype(int).astype(str)+'-01-01'

In [480]:
readme, f, fc = save(df,"fig4_job_quality",LOCAL)

,cohort,"% Full-time, permanent",% Different job,% Large firm,% Professional job,% On-the-job training,dt
0,2004.0,64.229918,76.599216,22.048545,12.091739,40.494147,2004-01-01
1,2005.0,63.071954,73.844385,22.032963,13.813077,39.357901,2005-01-01
2,2006.0,62.292421,75.072557,21.662642,13.871329,38.733107,2006-01-01
3,2007.0,64.411360,75.414491,21.540560,14.509934,39.952388,2007-01-01
4,2008.0,58.032721,67.232686,23.556279,13.693449,36.233839,2008-01-01


In [481]:
xaxis = alt.Chart(pd.DataFrame([{'x':'2004-01-01','y':0},{'x':'2021-01-01','y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "dt:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="Job quality measures by graduation cohort (%)",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        scale=alt.Scale(domain=[0, 100],clamp=True),
        
    )
)

ylabel=alt.Chart(pd.DataFrame([{'x':'2007-10-01','y':0,'t':'Global Financial Crisis'}]))\
    .mark_text(color=colors["eco-gray"],opacity=mo+0.1,align='left',dy=-10,dx=5).encode(x='x:T',y='y:Q',text='t:N')
ylabel2=alt.Chart(pd.DataFrame([{'x':'2020-03-01','y':0,'t':'COVID-19 →'}]))\
    .mark_text(color=colors["eco-gray"],opacity=mo+0.3,align='left',dy=-10,dx=5).encode(x='x:T',y='y:Q',text='t:N')

base = alt.Chart(fc).encode(x='dt:T')
area1 = alt.Chart(pd.DataFrame([{'x':'2007-11-01','y':0,'y2':100},{'x':'2011-10-01','y':0,'y2':100}])).mark_area(color=colors["eco-gray"],opacity=0.1)\
    .encode(x='x:T',y='y:Q',y2='y2:Q')
area2 = alt.Chart(pd.DataFrame([{'x':'2020-03-01','y':0,'y2':100},{'x':'2021-01-01','y':0,'y2':100}])).mark_area(color=colors["eco-gray"],opacity=0.1)\
    .encode(x='x:T',y='y:Q',y2='y2:Q')
area2e=alt.Chart(pd.DataFrame([{'x':'2021-01-01','y':0,'y2':100},{'x':'2023-01-01','y':0,'y2':100}])).mark_area(
    opacity=0.1,
    color=alt.Gradient(
        gradient='linear',
        stops=[alt.GradientStop(color='#ffffff00', offset=0),
               alt.GradientStop(color=colors["eco-gray"], offset=1)],
        x1=1,
        y1=1,
        x2=0,
        y2=1
        )
    ).encode(x='x:T',y='y:Q',y2='y2:Q')

line1=base.mark_line(color=colors['eco-dot'],interpolate='monotone').encode(y='% Large firm:Q')
line2=base.mark_line(color=colors['eco-light-blue'],interpolate='monotone').encode(y='% Full-time, permanent:Q')
line3=base.mark_line(color=colors['eco-turquiose'],interpolate='monotone').encode(y='% Different job:Q')
line4=base.mark_line(color=colors['eco-green'],interpolate='monotone').encode(y='% On-the-job training:Q')
line5=base.mark_line(color=colors['eco-yellow'],interpolate='monotone').encode(y='% Professional job:Q')

label1=line1.mark_text(color=colors['eco-dot'],align='left',dx=5,fontSize=12).encode(text='t:N').transform_calculate(t='"Large firm"').transform_filter('datum.dt>toDate("2020-03-01")')
label2=line2.mark_text(color=colors['eco-light-blue'],align='left',dx=5,fontSize=12).encode(text='t:N').transform_calculate(t='"Full-time, permanent"').transform_filter('datum.dt>toDate("2020-03-01")')
label3=line3.mark_text(color=colors['eco-turquiose'],align='left',dx=5,fontSize=12).encode(text='t:N').transform_calculate(t='"Different job"').transform_filter('datum.dt>toDate("2020-03-01")')
label4=line4.mark_text(color=colors['eco-green'],align='left',dx=5,fontSize=12).encode(text='t:N').transform_calculate(t='"On-the-job training"').transform_filter('datum.dt>toDate("2020-03-01")')
label5=line5.mark_text(color=colors['eco-yellow'],align='left',dx=5,fontSize=12).encode(text='t:N').transform_calculate(t='"Professional job"').transform_filter('datum.dt>toDate("2020-03-01")')
color=colors['eco-turquiose']
layer1 = (
    (xaxis+area1+area2+area2e+ylabel+ylabel2+line1+line2+line3+line4+line5+label1+label2+label3+label4+label5).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    open("README.md", "w").write(readme)

logo=alt.Chart(pd.DataFrame([{"x": '2023-01-01', "y": 100, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=80).encode(x='x:T',y='y:Q',url='img:N')
ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
    .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")

print(f+'\n')
layer1

WARN Infinite extent for field "dt": [Infinity, -Infinity]
WARN Infinite extent for field "dt": [Infinity, -Infinity]
WARN Infinite extent for field "dt": [Infinity, -Infinity]
WARN Infinite extent for field "dt": [Infinity, -Infinity]


fig4_job_quality



alt.LayerChart(...)